# Projeto Final - Ciência dos dados
### Alunos:
- Beatriz Lourenço 
- Fernando Kenji 
- Sophia Kerber
- Pedro H. Menezes

### Pergunta Motivadora:
É possível prever a avaliação de um livro (no ranking do Goodreads) com as seguintes variáveis?
- Inicial do autor
- Número de páginas
- Gênero
- Formato do livro (Capa dura, capa em papel, versão Kindle...)


### Por que regressão linear?


Como o objetivo do projeto era prever a avaliação do livro, decidiu-se utilizar a regressão linear para tanto, uma vez que a variável de saída é quantitativa. A ideia da regressão linear é aproximar os dados de uma reta, tentando diminuir o erro quadrático ao máximo. Utilizamos o One Hot Enconder, aliado ao SciKitLearn para dividir o dataset, classificar as variáveis qualitativas em números binários, e finalmente fazer a predição e o teste do dataset.

As técnicas que se prestam a este tipo de análise: regressão linear, *regression tree*, *random forest regression* - EXPLICAR OS OUTROS 2

### Como melhorar a análise?

Pode-se melhorar a análise feita por meio da K-Folds Cross Validation.

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [23]:
# Base de dados original
csv = pd.read_csv('book_data.csv')
csv.head()

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,Suzanne Collins,Winning will make you famous. Losing means cer...,NaN,Hardcover,9.78044E+12,374 pages,4.33,5519135,160706,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...
1,J.K. Rowling|Mary GrandPré,There is a door at the end of a silent corrido...,US Edition,Paperback,9.78044E+12,870 pages,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,Fantasy|Young Adult|Fiction,https://images.gr-assets.com/books/1255614970l...
2,Harper Lee,The unforgettable novel of a childhood in a sl...,50th Anniversary,Paperback,9.78006E+12,324 pages,4.27,3745197,79450,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...,https://images.gr-assets.com/books/1361975680l...
3,Jane Austen|Anna Quindlen|Mrs. Oliphant|George...,«È cosa ormai risaputa che a uno scapolo in po...,"Modern Library Classics, USA / CAN",Paperback,9.78068E+12,279 pages,4.25,2453620,54322,Pride and Prejudice,Classics|Fiction|Romance,https://images.gr-assets.com/books/1320399351l...
4,Stephenie Meyer,About three things I was absolutely positive.F...,NaN,Paperback,9.78032E+12,498 pages,3.58,4281268,97991,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...,https://images.gr-assets.com/books/1361039443l...


In [24]:
# Retirando os livros com informações faltando

csv.drop(['book_isbn','book_desc','book_edition','book_rating_count','book_review_count','image_url'],axis=1,inplace=True)
csv=csv.dropna()
len(csv.index)

48260

In [25]:
ar = csv['genres'].tolist()
ar1 = []
iniciais = []
paginas = []

# Escolhendo como gênero literário apenas o primeiro listado.
for i in ar:
    if i == "":
        ar1.append("")
    else:
        a = i.split('|')
        ar1.append(a[0])
        
# Selecionando apenas a inicial dos nomes dos autores
for i in csv['book_authors']:
    iniciais.append(i[0])

# Apenas o número de páginas aparecerá, sem a palavra "pages"
for i in csv['book_pages']:
    numero = i.split(' ')
    paginas.append(int(numero[0]))
print(paginas[0:10])
print(iniciais[0:10])
print(ar1[0:10])

[374, 870, 324, 279, 498, 552, 767, 122, 1037, 1728]
['S', 'J', 'H', 'J', 'S', 'M', 'C', 'G', 'M', 'J']
['Young Adult', 'Fantasy', 'Classics', 'Classics', 'Young Adult', 'Historical', 'Fantasy', 'Classics', 'Classics', 'Fantasy']


In [26]:
len(csv.index)

48260

In [27]:
# Criando colunas no DataFrame para as iniciais dos autores, os números de páginas e o principal gênero do livro
ar1=pd.DataFrame({'genero':ar1}, index=csv.index)
iniciais_ = pd.DataFrame({'iniciais':iniciais}, index=csv.index)
paginas_ = pd.DataFrame({'paginas':paginas}, index=csv.index)

In [37]:
csv = csv.join(ar1,how='inner')
csv = csv.join(iniciais_,how='inner')
csv = csv.join(paginas_,how='inner')

ValueError: columns overlap but no suffix specified: Index(['genero'], dtype='object')

In [38]:
csv.drop(['book_authors','genres','book_pages'],axis=1,inplace=True)


In [39]:
csv.head(15)

,book_format,book_rating,book_title,genero,iniciais,paginas
0,Hardcover,4.33,The Hunger Games,Young Adult,S,374
1,Paperback,4.48,Harry Potter and the Order of the Phoenix,Fantasy,J,870
2,Paperback,4.27,To Kill a Mockingbird,Classics,H,324
3,Paperback,4.25,Pride and Prejudice,Classics,J,279
4,Paperback,3.58,Twilight,Young Adult,S,498
5,Hardcover,4.36,The Book Thief,Historical,M,552
6,Paperback,4.25,The Chronicles of Narnia,Fantasy,C,767
7,Paperback,3.90,Animal Farm,Classics,G,122
8,Paperback,4.29,Gone with the Wind,Classics,M,1037
9,Mass Market Paperback,4.59,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,Fantasy,J,1728


In [40]:
csv['genero'].value_counts()

Fantasy                         7231
Fiction                         6554
Romance                         4029
Young Adult                     3582
Nonfiction                      2496
Historical                      2317
Mystery                         2019
Classics                        1948
Science Fiction                 1812
Sequential Art                  1450
Childrens                       1074
Horror                          1018
History                          876
Poetry                           822
Philosophy                       558
Paranormal                       554
Thriller                         527
Religion                         493
Christian Fiction                462
Short Stories                    451
Cultural                         434
Novels                           387
Biography                        385
New Adult                        354
Womens Fiction                   334
Christian                        319
Science                          318
E

In [41]:
csv.shape

(48260, 6)

In [42]:
# Apenas os livros com 3.5 estrelas ou mais de avaliação no Goodreads
csv.loc[(csv['book_rating']>= 3.5)]

,book_format,book_rating,book_title,genero,iniciais,paginas
0,Hardcover,4.33,The Hunger Games,Young Adult,S,374
1,Paperback,4.48,Harry Potter and the Order of the Phoenix,Fantasy,J,870
2,Paperback,4.27,To Kill a Mockingbird,Classics,H,324
3,Paperback,4.25,Pride and Prejudice,Classics,J,279
4,Paperback,3.58,Twilight,Young Adult,S,498
5,Hardcover,4.36,The Book Thief,Historical,M,552
6,Paperback,4.25,The Chronicles of Narnia,Fantasy,C,767
7,Paperback,3.90,Animal Farm,Classics,G,122
8,Paperback,4.29,Gone with the Wind,Classics,M,1037
9,Mass Market Paperback,4.59,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,Fantasy,J,1728


In [45]:
# Separando as variáveis em quantitativas, qualitativas e a desejada
y = csv['book_rating'].values
num = csv[['paginas']]
qual = csv[['book_format','genero','iniciais']]

In [46]:
qual.head()

,book_format,genero,iniciais
0,Hardcover,Young Adult,S
1,Paperback,Fantasy,J
2,Paperback,Classics,H
3,Paperback,Classics,J
4,Paperback,Young Adult,S


In [48]:
encoder = OneHotEncoder()
capa_encoded = encoder.fit_transform(qual)

In [49]:
x_num = num.values

In [50]:
#Transformar em array facilita a exploração dos dados
x_encoded = capa_encoded.toarray()

In [51]:
#Juntando a lista binária e as páginas
X = np.concatenate((x_num,x_encoded), axis=1)

In [52]:
X[0]

array([374.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   

In [53]:
#Rating do livro 1
y[0]

4.33

In [54]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing,utils, metrics
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

for i in range(len(y_pred)):
    if y_pred[i]>5:
        y_pred[i]=5
    elif y_pred[i]<0:
        y_pred[i]=0
        
df = pd.DataFrame({'Atual': y_test.flatten(), 'Predito': y_pred.flatten()})
df.head()

,Atual,Predito
0,3.86,3.863825
1,4.36,4.094461
2,4.52,4.179883
3,4.25,3.982733
4,3.57,3.843774


In [55]:
#Usando outro DataFrame (de 2018) para fazer a predição:
teste_X2 = pd.read_csv('book_data_2018.csv')

In [56]:
teste_X2=teste_X.dropna()
teste_X2.head()

NameError: name 'teste_X' is not defined

In [ ]:
y_pred2 = regressor.predict(teste_X2)

for i in range(len(y_pred2)):
    if y_pred2[i]>5:
        y_pred2[i]=5
    elif y_pred2[i]<0:
        y_pred2[i]=0
        
df2 = pd.DataFrame({'Atual': y_test2.flatten(), 'Predito': y_pred2.flatten()})
df2.head()

In [ ]:
df1 = df
df1.plot(kind='bar',figsize=(16,10))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.xlabel('Book Index')
plt.ylabel('Rating')
plt.show()

In [ ]:
print("O R² é " + str(metrics.r2_score(df['Atual'],df['Predito'])))